# End-to-end Reusable ML Pipeline with Seldon and Kubeflow

In this example we showcase how to build re-usable components to build an ML pipeline that can be trained and deployed at scale.

We will automate content moderation on the Reddit comments in /r/science building a machine learning NLP model with the following components:

![](img/completed-pipeline-deploy.jpg)

This tutorial will break down in the following sections:

1) Run all the services (Kubeflow and Seldon)

2) Test and build all our reusable pipeline steps

3) Use Kubeflow to Train the Pipeline and Deploy to Seldon

5) Test Seldon Deployed ML REST Endpoints

6) Visualise Seldon's Production ML Pipelines

## Before you start
Make sure you install the following dependencies, as they are critical for this example to work:

* Helm v3.0.0+
* A Kubernetes cluster running v1.13 or above (minkube / docker-for-windows work well if enough RAM)
* kubectl v1.14+
* ksonnet v0.13.1+
* kfctl 0.5.1 - Please use this exact version as there are major changes every few months
* Python 3.6+
* Python DEV requirements (we'll install them below)

Let's get started! 🚀🔥 We will be building the end-to-end pipeline below:

![](img/kubeflow-seldon-nlp-full.jpg)


## Notes from Rafal: general

* I did run it with Kubernetes 1.14 on Digital Ocean because of various incompatibilities between Kubernetes, Kubeflow and Ksonnet documented [here](https://www.kubeflow.org/docs/started/k8s/overview/) and [here](https://github.com/kubeflow/kubeflow/issues/3544).
* I use images pushed to DockerHub, see [this](./push-docker.sh) script.
* I use port forwarding to access services on my localhost
* Make sure to install seldon-core on the cluster using [seldon_core_setup](../seldon_core_setup.ipynb) notebook

In [ ]:
!cat requirements-dev.txt

In [ ]:
!pip install --user -r requirements-dev.txt

## 1) Run all the services (Kubeflow and Seldon)
Kubeflow's CLI allows us to create a project which will allow us to build the configuration we need to deploy our kubeflow and seldon clusters.

In [ ]:
!kfctl init kubeflow-seldon
!ls kubeflow-seldon

Now we run the following commands to basically launch our Kubeflow cluster with all its components. 

It may take a while to download all the images for Kubeflow so feel free to make yourself a cup of ☕.

If you have a terminal you can see how the containers are created in real-time by running `kubectl get pods -n kubeflow -w`.

In [ ]:
%%bash
cd kubeflow-seldon
kfctl generate all -V

In [ ]:
%%bash
cd kubeflow-seldon
kfctl apply all -V

### Temporary fix for Argo image

At the time of writing we need to make some updates in the Argo images with the following commands below.

(This basically changes the images to the latest ones, otherwise we will get an error when we attach the volume)


In [ ]:
!kubectl -n kubeflow patch deployments. workflow-controller --patch '{"spec": {"template": {"spec": {"containers": [{"name": "workflow-controller", "image": "argoproj/workflow-controller:v2.3.0-rc3"}]}}}}'
!kubectl -n kubeflow patch deployments. ml-pipeline --patch '{"spec": {"template": {"spec": {"containers": [{"name": "ml-pipeline-api-server", "image": "elikatsis/ml-pipeline-api-server:0.1.18-pick-1289"}]}}}}'
# !kubectl -n kubeflow patch configmaps workflow-controller-configmap --patch '{"data": {"config": "{ executorImage: argoproj/argoexec:v2.3.0-rc3,artifactRepository:{s3: {bucket: mlpipeline,keyPrefix: artifacts,endpoint: minio-service.kubeflow:9000,insecure: true,accessKeySecret: {name: mlpipeline-minio-artifact,key: accesskey},secretKeySecret: {name: mlpipeline-minio-artifact,key: secretkey}}}}" }}'

The last command you need to run actually needs to be manual as the patch cannot change configmap contents directly

You need to run the edit commad and change the executorImage to: `argoproj/argoexec:v2.3.0-rc3`

The command should be run from a terminal:

```
kubectl edit configmaps workflow-controller-configmap -n kubeflow
```

## Notes from Rafal: PVC fixes for Digital Ocean

1. Add `get` verb to `persistentvolumeclaims` as described [here](https://github.com/kubeflow/pipelines/issues/1482#issuecomment-507740533) with `kubectl -n kubeflow edit clusterrole pipeline-runner`
2. Note in [nlp_pipeline.py](train_pipeline/nlp_pipeline.py) change of `ReadWriteMany` to `ReadWriteOnce`

## 2) Test and build all our reusable pipeline steps

We will start by building each of the components in our ML pipeline. 

![](img/kubeflow-seldon-nlp-reusable-components.jpg)

### Let's first have a look at our clean_text step:


In [ ]:
!ls pipeline/pipeline_steps

Like in this step, all of the other steps can be found in the `pipeline/pipeline_steps/` folder, and all have the following structure:
* `pipeline_step.py` which exposes the functionality through a CLI 
* `Transformer.py` which transforms the data accordingly
* `requirements.txt` which states the python dependencies to run
* `build_image.sh` which uses `s2i` to build the image with one line

### Let's check out the CLI for clean_text
The pipeline_step CLI is the entry point for the kubeflow image as it will be able to pass any relevant parameters


In [ ]:
!python3 pipeline/pipeline_steps/clean_text/pipeline_step.py --help

This is actually a very simple file, as we are using the click library to define the commands:


In [ ]:
!cat pipeline/pipeline_steps/clean_text/pipeline_step.py

The Transformer is where the data munging and transformation stage comes in, which will be wrapped by the container and exposed through the Seldon Engine to ensure our pipeline can be used in production.

Seldon provides multiple different features, such as abilities to send custom metrics, pre-process / post-process data and more. In this example we will only be exposing the `predict` step.

In [ ]:
!cat pipeline/pipeline_steps/clean_text/Transformer.py

If you want to understand how the CLI pipeline talks to each other, have a look at the end to end test in `pipeline/pipeline_tests/`:

In [ ]:
# !pytest ./pipeline/pipeline_tests/. --disable-pytest-warnings

To build the image we provide a build script in each of the steps that contains the instructions:

In [ ]:
!cat pipeline/pipeline_steps/clean_text/build_image.sh

The only thing you need to make sure is that Seldon knows how to wrap the right model and file.

This can be achieved with the s2i/environment file. 

As you can see, here we just tell it we want it to use our `Transformer.py` file:


In [ ]:
!cat pipeline/pipeline_steps/clean_text/.s2i/environment

Once this is defined, the only thing we need to do is to run the `build_image.sh` for all the reusable components.

Here we show the manual way to do it: 

In [ ]:
# %%bash
# # we must be in the same directory
# cd pipeline/pipeline_steps/clean_text/ && ./build_image.sh
# cd ../data_downloader && ./build_image.sh
# cd ../lr_text_classifier && ./build_image.sh
# cd ../spacy_tokenize && ./build_image.sh
# cd ../tfidf_vectorizer && ./build_image.sh

### Note from Rafal: 
I built it manually and uploaded to docker hub, commenting to not rebuild by accident without reason

## 3) Train our NLP Pipeline through the Kubeflow UI
We can access the Kubeflow dashboard to train our ML pipeline via http://localhost/_/pipeline-dashboard

If you can't edit this, you need to make sure that the ambassador gateway service is accessible:


In [ ]:
!kubectl get svc ambassador -n kubeflow

In my case, I need to change the kind from `NodePort` into `LoadBalancer` which can be done with the following command:

(note from Rafal: not using LoadBalancer as it exposes service without password publicly, need to look into it)

In [ ]:
# !kubectl patch svc ambassador --type='json' -p '[{"op":"replace","path":"/spec/type","value":"LoadBalancer"}]' -n kubeflow
!kubectl patch svc ambassador --type='json' -p '[{"op":"replace","path":"/spec/type","value":"NodePort"}]' -n kubeflow

In [ ]:
!kubectl get svc ambassador -n kubeflow

To forward ambassador port instead execute in terminal:
```bash
kubectl port-forward svc/ambassador 8000:80 -n kubeflow
```

Now that I've changed it to a loadbalancer, it has allocated the external IP as my localhost so I can access it at http://localhost:8000/_/pipeline-dashboard


If this was successfull, you should be able to access the dashboard
![](img/k-pipeline-dashboard.jpg)

### Define the pipeline
Now we want to generate the pipeline. For this we can use the DSL provided by kubeflow to define the actual steps required. 

The pipeline will look as follows:

![](img/kubeflow-seldon-nlp-ml-pipelines.jpg)

In [ ]:
!cat train_pipeline/nlp_pipeline.py

### Breaking down the  code
As you can see in the DSL, we have the ContainerOp - each of those is a step in the Kubeflow pipeline.

At the end we can see the `seldondeploy` step which basically deploys the trained pipeline

The definition of the SeldonDeployment graph is provided in the `deploy_pipeline/seldon_production_pipeline.yaml` file.

The seldondeployment file defines our production execution graph using the same reusable components.

In [ ]:
!cat deploy_pipeline/seldon_production_pipeline.yaml

### Seldon Production pipeline contents
If we look at the file we'll be using to deploy our pipeline, we can see that it has the following key points:

1) Reusable components definitions as containerSpecs: cleantext, spacytokenizer, tfidfvectorizer & lrclassifier

2) DAG (directed acyclic graph) definition for REST pipeline: cleantext -> spacytokenizer -> tfidfvectorizer -> lrclassifier

This graph in our production deployment looks as follows:

![](img/kubeflow-seldon-nlp-ml-pipelines-deploy.jpg)

### Generate the pipeline files to upload to Kubeflow
To generate the pipeline we just have to run the pipeline file, which will output the `tar.gz` file that will be uploaded.

In [ ]:
%%bash
# Generating graph definition
python3 train_pipeline/nlp_pipeline.py
ls train_pipeline/

### Note from Rafal: apply following so seldon core deployments work

```
kubectl create clusterrolebinding pipelinerunnerbinding \
  --clusterrole=cluster-admin \
  --serviceaccount=kubeflow:pipeline-runner
```


### Run the pipeline

We now need to upload the resulting `nlp_pipeline.py.tar.gz` file generated.

This can be done through the "Upload PIpeline" button in the UI at http://localhost:8000/_/pipeline-dashboard.

Once it's uploaded, we want to create and trigger a run! You should now be able to see how each step is executed:

![](img/running-pipeline.jpg)

### Inspecting the data created in the Persistent Volume
The pipeline saves the output of the pipeline together with the trained model in the persistent volume claim.

The persistent volume claim is the same name as the argo workflow:

In [ ]:
!kubectl get workflow -n kubeflow

Our workflow is there! So we can actually access it by running

In [ ]:
!kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}'

And we can use good old `sed` to insert this workflow name in our PVC-Access controler which we can use to inspect the contents of the volume:

In [ ]:
!sed "s/PVC_NAME/"$(kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}')"-my-pvc/g" deploy_pipeline/pvc-access.yaml 

We just need to apply this container with our kubectl command, and we can use it to inspect the mounted folder:

In [ ]:
!sed "s/PVC_NAME/"$(kubectl get workflow -n kubeflow -o jsonpath='{.items[0].metadata.name}')"-my-pvc/g" deploy_pipeline/pvc-access.yaml | kubectl -n kubeflow apply -f -

In [ ]:
!kubectl get pods -n kubeflow pvc-access-container

Now we can run an `ls` command to see what's inside:

In [ ]:
!kubectl -n kubeflow exec -it pvc-access-container ls /mnt

In [ ]:
!kubectl delete -f deploy_pipeline/pvc-access.yaml -n kubeflow

## 5) Test Deployed ML REST Endpoints
Now that it's running we have a production ML text pipeline that we can Query using REST and GRPC


First we can check if our Seldon deployment is running with

In [ ]:
!kubectl -n kubeflow get seldondeployment 

We will need the Seldon Pipeline Deployment name to reach the API, so we can get it using:

In [ ]:
!kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}'

Now we can interact with our API in two ways: 

1) Using CURL or any client like PostMan

2) Using the Python SeldonClient

### Using CURL from the terminal
When using CURL, the only thing we need to provide is the data in JSON format, as well as the url, which is of the format:

```
http://<ENDPOINT>/seldon/kubeflow/<PIPELINE_NAME>/api/v0.1/predictions
```

In [ ]:
%%bash
curl -v -X POST -H 'Content-Type: application/json' \
    -d "{'data': {'names': ['text'], 'ndarray': ['Hello world this is a test']}}" \
    http://127.0.0.1:8000/seldon/kubeflow/$(kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}')/api/v0.1/predictions

### Using the SeldonClient
We can also use the Python SeldonClient to interact with the pipeline we just deployed 

In [ ]:
from seldon_core.seldon_client import SeldonClient
import numpy as np
import subprocess

host = "localhost"
port = "8000" # Make sure you use the port above
batch = np.array(["Hello world this is a test"])
payload_type = "ndarray"
# Get the deployment name
deployment_name = subprocess.getoutput("kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}'")
transport="rest"
namespace="kubeflow"

sc = SeldonClient(
    gateway="ambassador", 
    gateway_endpoint=host + ":" + port,
    namespace=namespace)

client_prediction = sc.predict(
    data=batch, 
    deployment_name=deployment_name,
    names=["text"],
    payload_type=payload_type,
    transport="rest")

print(client_prediction)

## 6) Visualise Seldon's Production ML Pipelines
We can visualise the performance using the SeldonAnalytics package, which we can deploy using:

In [ ]:
!helm install seldon-core-analytics seldon-core-analytics --repo https://storage.googleapis.com/seldon-charts --namespace kubeflow

In my case, similar to what I did with Ambassador, I need to make sure the the service is a LoadBalancer instead of a NodePort

In [ ]:
!kubectl patch svc grafana-prom --type='json' -p '[{"op":"replace","path":"/spec/type","value":"LoadBalancer"}]' -n kubeflow

In [ ]:
!kubectl get svc grafana-prom -n kubeflow

Now we can access it at the port provided, in my case it is http://localhost:32445/d/3swM2iGWz/prediction-analytics?refresh=5s&orgId=1

(initial username is admin and password is password, which will be requested to be changed on the first login)

### Note from Rafal: use port forwarding

Forward ports from Grafana with
```bash
kubectl port-forward svc/grafana-prom 8001:80 -n kubeflow
```
and go to http://localhost:8001

Generate a bunch of requests and visualise:

In [ ]:
while True:
    client_prediction = sc.predict(
        data=batch, 
        deployment_name=deployment_name,
        names=["text"],
        payload_type=payload_type,
        transport="rest")

## You now have a full end-to-end training and production NLP pipeline 😎 
![](img/seldon-analytics.jpg)